<a href="https://colab.research.google.com/github/MariannaMois/ml_sellout_marketplace/blob/master/name_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Проект по анализу и унификации данных о продажах (маркетплейс Amazon)**

##Введение
В условиях высокой конкуренции на маркетплейсах продавцы часто используют вариативные названия для одинаковых товаров, что приводит к:

* искажению аналитики продаж

* трудностям в идентификации популярных товаров

* ошибкам в прогнозировании спроса

* низкой эффективности рекламных кампаний

Проект направлен на решение проблемы группировки товаров с разными названиями, но идентичными характеристиками (бренд, тип продукта, линейка).

Задачи проекта
1. *Предобработка данных*

* очистка текста от спецсимволов и цифр

* удаление стоп-слов и маркетинговых конструкций

* стандартизация регистра

2. *Лингвистический анализ*

* извлечение ключевых слов (NLP)

* распознавание именованных сущностей (NER)

* выделение бренда и базовых характеристик

3. *Идентификация продуктов*

* сравнение строк с использованием fuzzy-логики


# Технологический стек

| Компонент          | Технологии/Инструменты               |
|--------------------|--------------------------------------|
| Обработка данных   | Pandas, NumPy                        |
| NLP                | spaCy, Transformers                 |
| Сравнение строк    | FuzzyWuzzy (fuzz.ratio)              |
| Инфраструктура     | Google Colab                         |

# Подключение и установка библиотек


На данном этапе подключается Google Drive для работы с файлами, а также устанавливаются библиотеки для обработки текста и работы с Excel.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=b04dd87f7fa58e31a379bd40695e4b5b54aa5109f2a6e78ea4253deeb47e10a6
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [ ]:
!pip install spacy
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np

#Предобработка данных

In [ ]:
file_path = '/content/drive/MyDrive/test amazon.csv'

test = pd.read_csv(file_path, delimiter=',')

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   group          536 non-null    object 
 1   type           536 non-null    object 
 2   product        536 non-null    object 
 3   brand          398 non-null    object 
 4   asin           536 non-null    object 
 5   date           536 non-null    object 
 6   Сумма sales    536 non-null    int64  
 7   Сумма revenue  536 non-null    float64
 8   Медиана price  536 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 37.8+ KB


In [ ]:
test['product'] = test['product'].str.replace('[|()$]', '', regex=True)

## Лингвистический анализ

*Пробую несколько методов, после тестирования можно оставить один*


Для проекта не было готового решения, которое можно было бы адаптировать.

**Стратегия**

**Извлечение бренда**:


Бренд часто стоит первым словом в строке и пишется в верхнем регистре (например, BUXOM, LAWLESS).

Метод: правило "первое слово в верхнем регистре" или POS-тег PROPN (имя собственные).

**Извлечение названия линейки:**

Название обычно идет после бренда и до первого стоп-символа (,, ;, -).

Пример: "Lash Volumizing Mascara" в строке BUXOM Lash Volumizing Mascara.

В столбце short_name_spacy будут сохранены названия полученные с помощью POS-тег, без удаления прилагательных (это не всегда "шум", может быть частично название самого продукта)



In [24]:
import spacy
import re

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Функция для извлечения ключевых слов с помощью spaCy
def shorten_with_spacy(text):
    doc = nlp(text)
    # Извлекаем существительные, имена собственные и прилагательные (как ключевые слова)
    keywords = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN", "ADJ"]]
    return " ".join(keywords)

In [ ]:
test['short_name_spacy'] = test['product'].apply(shorten_with_spacy)

In [ ]:
test

,group,type,product,brand,asin,date,Сумма sales,Сумма revenue,Медиана price,short_name_spacy
0,Глаза,тушь,2 in 1 Mascara for VIBELY Mascara 5x Longer W...,NaN,B0BP8D14J1,2024-11-01 00:00:00,2040,14259.60,6.990000,Mascara VIBELY Mascara Longer Waterproof Lash ...
1,Глаза,тушь,"2 Pack 2 in 1 QIC Mascara, 5x Longer Washable...",NaN,B0CYM2S6G2,2024-11-01 00:00:00,279,3624.21,12.990000,Pack QIC Mascara Longer Washable Black Mascara...
2,Глаза,тушь,"2 Pack 2 in 1 QIC Mascara, 5x Longer Washable...",NaN,B0CYM2S6G2,2024-12-01 00:00:00,259,3026.34,11.684710,Pack QIC Mascara Longer Washable Black Mascara...
3,Глаза,тушь,"4 Pack VIBELY Mascara 5x Longer Washable, 2 i...",NaN,B0BYW9VB1V,2024-11-01 00:00:00,2040,34659.60,16.990000,Pack VIBELY Mascara Washable 4D Silk Fiber Las...
4,Глаза,тушь,"4D Silk Fiber Lash Mascara, Waterproof Tubing...",NaN,B07ZWW7G71,2024-11-01 00:00:00,2831,22619.69,7.990000,4D Silk Fiber Lash Mascara Waterproof Tubing M...
...,...,...,...,...,...,...,...,...,...,...
531,Глаза,тушь,wet n wild Mega Clear Brow & Lash Mascara - Sc...,WET N WILD,B09NXPG6D5,2024-11-01 00:00:00,9680,22070.40,2.280000,wet wild Mega Clear Brow Lash Mascara Sculpts ...
532,Глаза,тушь,"wet n wild Mega Length, Vitamin E Enriched Len...",WET N WILD,B07MDTVGGL,2024-11-01 00:00:00,3259,10689.52,3.280000,wet wild Mega Length Vitamin E Enriched Length...
533,Глаза,тушь,"wet n wild Mega Length, Vitamin E Enriched Len...",WET N WILD,B07MDTVGGL,2024-12-01 00:00:00,5030,13881.78,2.759797,wet wild Mega Length Vitamin E Enriched Length...
534,Глаза,тушь,"Youngfocus 3D Fiber Lash Mascara Waterproof, L...",YOUNGFOCUS,B06XBQKV2M,2024-12-01 00:00:00,500,8499.44,16.998880,Youngfocus 3D Fiber Lash Mascara Waterproof Lo...


Составим функцию для чистки от "шума", которую можно использовать и для исходных данных и для названий после обработки для удаления точечных ошибок

In [28]:
def extract_short_name(text):
    # Убираем лишние символы и цифры, если они есть
    text = re.sub(r"$\$$|\d+|\b(?:a|an|the|and|or|of|in|on|at|by|for|with|to|pack|from|!|x|3D|4D|5D|2 in 1|5x|flake free|'| d |%|™ |)\b", "", text.lower()).strip()

    # Регулярное выражение для поиска бренда и основного названия продукта
    match = re.search(
        r"((?:\d+D\s)?(?:Fiber\s)?(?:Lash\s)?(?:Mascara\s)?(?:Kit\s)?)"  # Название продукта
        r".*?"  # Любые символы
        r"(by\s|from\s)?([A-Za-zÉéèÈëËöÖüÜäÄçÇ&\- ]+)"  # Бренд
        r"|"  # ИЛИ
        r"([A-Za-zÉéèÈëËöÖüÜäÄçÇ&\- ]+)\s"  # Бренд в начале
        r"((?:\d+D\s)?(?:Fiber\s)?(?:Lash\s)?(?:Mascara\s)?(?:Kit\s)?)",  # Название продукта
        text,
        re.IGNORECASE  # Игнорируем регистр для более гибкого поиска
    )

    if match:
        # Если бренд в конце текста (например, "by Simply Naked Beauty")
        if match.group(3):
            brand = match.group(3).strip()
            product_name = match.group(1).strip()
        # Если бренд в начале текста
        elif match.group(4):
            brand = match.group(4).strip()
            product_name = match.group(5).strip()
        else:
            return ""

        return f"{brand} {product_name}"

    # Если не удалось найти совпадение, возвращаем исходный текст (или пустую строку)
    return ""

In [29]:
test['short_product_name'] = test['short_name_spacy'].apply(extract_short_name)

In [30]:
test

,group,type,product,brand,asin,date,Сумма sales,Сумма revenue,Медиана price,short_name_spacy,short_product_name
0,Глаза,тушь,2 in 1 Mascara for VIBELY Mascara 5x Longer W...,NaN,B0BP8D14J1,2024-11-01 00:00:00,2040,14259.60,6.990000,Mascara VIBELY Mascara Longer Waterproof Lash ...,vibely mascara longer waterproof lash cosmetic...
1,Глаза,тушь,"2 Pack 2 in 1 QIC Mascara, 5x Longer Washable...",NaN,B0CYM2S6G2,2024-11-01 00:00:00,279,3624.21,12.990000,Pack QIC Mascara Longer Washable Black Mascara...,qic mascara longer washable black mascara d si...
2,Глаза,тушь,"2 Pack 2 in 1 QIC Mascara, 5x Longer Washable...",NaN,B0CYM2S6G2,2024-12-01 00:00:00,259,3026.34,11.684710,Pack QIC Mascara Longer Washable Black Mascara...,qic mascara longer washable black mascara d si...
3,Глаза,тушь,"4 Pack VIBELY Mascara 5x Longer Washable, 2 i...",NaN,B0BYW9VB1V,2024-11-01 00:00:00,2040,34659.60,16.990000,Pack VIBELY Mascara Washable 4D Silk Fiber Las...,vibely mascara washable d silk fiber lash masc...
4,Глаза,тушь,"4D Silk Fiber Lash Mascara, Waterproof Tubing...",NaN,B07ZWW7G71,2024-11-01 00:00:00,2831,22619.69,7.990000,4D Silk Fiber Lash Mascara Waterproof Tubing M...,d silk fiber lash mascara waterproof tubing ma...
...,...,...,...,...,...,...,...,...,...,...,...
531,Глаза,тушь,wet n wild Mega Clear Brow & Lash Mascara - Sc...,WET N WILD,B09NXPG6D5,2024-11-01 00:00:00,9680,22070.40,2.280000,wet wild Mega Clear Brow Lash Mascara Sculpts ...,wet wild mega clear brow lash mascara sculpts ...
532,Глаза,тушь,"wet n wild Mega Length, Vitamin E Enriched Len...",WET N WILD,B07MDTVGGL,2024-11-01 00:00:00,3259,10689.52,3.280000,wet wild Mega Length Vitamin E Enriched Length...,wet wild mega length vitamin e enriched length...
533,Глаза,тушь,"wet n wild Mega Length, Vitamin E Enriched Len...",WET N WILD,B07MDTVGGL,2024-12-01 00:00:00,5030,13881.78,2.759797,wet wild Mega Length Vitamin E Enriched Length...,wet wild mega length vitamin e enriched length...
534,Глаза,тушь,"Youngfocus 3D Fiber Lash Mascara Waterproof, L...",YOUNGFOCUS,B06XBQKV2M,2024-12-01 00:00:00,500,8499.44,16.998880,Youngfocus 3D Fiber Lash Mascara Waterproof Lo...,youngfocus d fiber lash mascara waterproof lon...


Сравнение по неточному совпадению коротких названий, обязательное условие совпадение по бренду

Наименование SCU условное, не опечатка от SKU - можно назвать по-другому, но не дублировать "SKU" или ASIN, так как эти столбцы могут быть в исходном датасете.

In [31]:
!pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.8 MB/s eta 0:00:00


In [32]:
from fuzzywuzzy import fuzz

In [43]:
test['SCU'] = None

In [44]:
for i in range(len(test)):
    # если short_product_name не пустое и содержит (длина) более 1 слов
    if pd.notna(test.loc[i, 'short_product_name']) and len(test.loc[i, 'short_product_name'].split()) > 1:
        for j in range(i + 1, len(test)):
            # если бренды совпадают! можно еще добавить категории
            if test.loc[i, 'brand'] == test.loc[j, 'brand']:
                # сравнение текстовых описаний, условно 50% схожести
                if fuzz.token_sort_ratio(test.loc[i, 'short_product_name'], test.loc[j, 'short_product_name']) >= 50:
                    # значение short_product_name в новый столбец SCU
                    test.loc[i, 'SCU'] = test.loc[i, 'short_product_name']
                    test.loc[j, 'SCU'] = test.loc[i, 'short_product_name']

In [46]:
# Заполняем пропуски в SCU
test['SCU'] = test['SCU'].fillna(
    test['short_product_name'].str.lower()  # Берем значения из short_name_spacy и приводим к нижнему регистру
)

In [47]:
test

,group,type,product,brand,asin,date,Сумма sales,Сумма revenue,Медиана price,short_name_spacy,short_product_name,SCU
0,Глаза,тушь,2 in 1 Mascara for VIBELY Mascara 5x Longer W...,NaN,B0BP8D14J1,2024-11-01 00:00:00,2040,14259.60,6.990000,Mascara VIBELY Mascara Longer Waterproof Lash ...,vibely mascara longer waterproof lash cosmetic...,vibely mascara longer waterproof lash cosmetic...
1,Глаза,тушь,"2 Pack 2 in 1 QIC Mascara, 5x Longer Washable...",NaN,B0CYM2S6G2,2024-11-01 00:00:00,279,3624.21,12.990000,Pack QIC Mascara Longer Washable Black Mascara...,qic mascara longer washable black mascara d si...,qic mascara longer washable black mascara d si...
2,Глаза,тушь,"2 Pack 2 in 1 QIC Mascara, 5x Longer Washable...",NaN,B0CYM2S6G2,2024-12-01 00:00:00,259,3026.34,11.684710,Pack QIC Mascara Longer Washable Black Mascara...,qic mascara longer washable black mascara d si...,qic mascara longer washable black mascara d si...
3,Глаза,тушь,"4 Pack VIBELY Mascara 5x Longer Washable, 2 i...",NaN,B0BYW9VB1V,2024-11-01 00:00:00,2040,34659.60,16.990000,Pack VIBELY Mascara Washable 4D Silk Fiber Las...,vibely mascara washable d silk fiber lash masc...,vibely mascara washable d silk fiber lash masc...
4,Глаза,тушь,"4D Silk Fiber Lash Mascara, Waterproof Tubing...",NaN,B07ZWW7G71,2024-11-01 00:00:00,2831,22619.69,7.990000,4D Silk Fiber Lash Mascara Waterproof Tubing M...,d silk fiber lash mascara waterproof tubing ma...,d silk fiber lash mascara waterproof tubing ma...
...,...,...,...,...,...,...,...,...,...,...,...,...
531,Глаза,тушь,wet n wild Mega Clear Brow & Lash Mascara - Sc...,WET N WILD,B09NXPG6D5,2024-11-01 00:00:00,9680,22070.40,2.280000,wet wild Mega Clear Brow Lash Mascara Sculpts ...,wet wild mega clear brow lash mascara sculpts ...,wet wild mega clear brow lash mascara sculpts ...
532,Глаза,тушь,"wet n wild Mega Length, Vitamin E Enriched Len...",WET N WILD,B07MDTVGGL,2024-11-01 00:00:00,3259,10689.52,3.280000,wet wild Mega Length Vitamin E Enriched Length...,wet wild mega length vitamin e enriched length...,wet wild mega length vitamin e enriched length...
533,Глаза,тушь,"wet n wild Mega Length, Vitamin E Enriched Len...",WET N WILD,B07MDTVGGL,2024-12-01 00:00:00,5030,13881.78,2.759797,wet wild Mega Length Vitamin E Enriched Length...,wet wild mega length vitamin e enriched length...,wet wild mega length vitamin e enriched length...
534,Глаза,тушь,"Youngfocus 3D Fiber Lash Mascara Waterproof, L...",YOUNGFOCUS,B06XBQKV2M,2024-12-01 00:00:00,500,8499.44,16.998880,Youngfocus 3D Fiber Lash Mascara Waterproof Lo...,youngfocus d fiber lash mascara waterproof lon...,youngfocus d fiber lash mascara waterproof lon...


In [48]:
test.to_excel("amazon.xlsx", index=False, engine='openpyxl')

In [49]:
try:
    from google.colab import files
    files.download('amazon.xlsx')
except ImportError:
    print("Функция загрузки файла не поддерживается в вашей среде")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>